# BioExplorer - Astrocytes
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load astrocytes

In [ ]:
assembly_name = 'Astrocytes'
be.remove_assembly(assembly_name)
vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_astrocytes(
    assembly_name=assembly_name,
    population_name='ngv', vasculature_population_name='Vasculature',
    population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    radius_multiplier=1.0,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True, generate_internals=False,
    load_somas=True, load_dendrites=True, load_end_feet=True,
    # sql_filter='x>500 AND x<600 AND y>1500 AND y<1600 AND z>500 AND z<600'
    # sql_filter='guid % 10=0'
    # sql_filter='guid=7178'
)

In [ ]:
print(be.reset_camera())
core.set_renderer(current='basic', subsampling=4, max_accum_frames=4)

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, be.NB_MATERIALS_PER_MORPHOLOGY)

    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        color = palette[mid]
        if mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            colors.append(color)
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.2)
            user_params.append(0.01)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.001)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

import seaborn as sns
model_ids = be.get_model_ids()['ids']
nb_models = len(model_ids)
for model_id in model_ids:
    set_morphology_materials(model_id, 'Set3')
core.set_renderer()

In [ ]:
print(be.reset_camera())
core.set_renderer()

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
mm.create_snapshot(
    renderer='bio_explorer',
    path='/tmp', base_name='vaculature_astrocytes_cartoon_1k_v1',
    size=[1000, 1000], samples_per_pixel=64)